# 为什么要进行 Normalization ?

本网络层输出值的方差与上一网络层的神经元个数、上一网络层的输出值（亦即本网络层的输入值）的方差、上一网络层输出值的权值的方差有关，为了控制本网络层输出值的方差，我们可以通过权值初始化来控制上一网络层输出值的权值的方差（即 $D(W)$），除此之外，我们也可以通过 Normalization 来控制上一网络层的输出值（亦即本网络层的输入值）的方差（即 $D(X)$）。

![](./图片1.png)

# Batch Normalization

**1. 批标准化 Batch Normalization：**
- 批是指一批数据，通常为 mini-batch；
- 标准化是处理后的数据服从 $N(0,1)$ 的标准正态分布。

<br/>

**2. Batch Normalization 最初想要解决的问题**：在训练过程中，数据需要经过多层的网络，如果数据在前向传播的过程中，数据尺度发生了变化，可能会导致梯度爆炸或者梯度消失，从而导致模型无法训练。

<br/>

**3. 批标准化带来的额外优点：**
- 可以使用更大的学习率，加速模型收敛
- 可以不用精心设计权值初始化（有了 BN 层就可以不用精心设计权值初始化，可以将权值直接初始化为标准正态分布）
- 可以不用 dropout 或者较小的 dropout
- 可以不用 L2 或者可以使用较小的 L2(weight decay)
- 可以不用 LRN (local response normalization)

<br/>

**4. Batch Normalization 的运算过程：**

**标准化**：$\widehat{x}_{i} \leftarrow \frac{x{i}-\mu_{\mathcal{B}}}{\sqrt{\sigma_{B}^{2}+\epsilon}}$，其中$\epsilon$ 是一个调整项

**affine transform(缩放和平移)**：${y}_{i} \leftarrow \gamma \widehat{x}_{i}+\beta \equiv \mathrm{B} \mathrm{N}_{\gamma,\beta}\left(x_{i}\right)$，这个操作可以增强模型的 capacity，也就是让模型自己判断是否要对数据进行标准化，进行多大程度的标准化。如果$\gamma= \sqrt{\sigma_{B}^{2}}$，$\beta=\mu_{\mathcal{B}}$，那么就实现了${x}_{i} \rightarrow {y}_{i}$ 的恒等映射。

**观察 Batch Normalization 的运算原理**：

我们可以通过下图看到，Batch Normalization 在每一个特征维度上计算所有样本（样本数量为 batch_size 大小）的 runninng_mean 均值、running_var 方差、weight 和 bias。 

![](./图片2.png)

<br/>

**5. Batch Normalization 的主要属性：**
- runninng_mean：均值
- running_var：方差
- weight：affine transform 中的 $\gamma$
- bias：affine transform 中的 $\beta$

weight 和 bias 是训练中可学习的参数

**观察 weight 和 bias 的 shape**：

weight 和 bias 的 shape 都等于特征的数量 num_features

In [1]:
import torch
import torch.nn as nn

# 假设输入数据的形状是 (batch_size, num_features, features_channels, features_length, features_width)，创建一个输入张量
batch_size = 2
num_features = 4
features_channels = 3
features_length = 2
features_width = 2
input_tensor = torch.randn(batch_size, num_features, features_channels, features_length, features_width)

# 创建一个 BatchNorm3d 模块
bn = nn.BatchNorm3d(num_features)

# 应用 BatchNorm3d
normalized_tensor = bn(input_tensor)

print(bn.weight.shape)
print(bn.bias.shape)

torch.Size([4])
torch.Size([4])


<br/>

**6. 以 `nn.BatchNorm3d()` 为例，介绍参数：**

`torch.nn.BatchNorm3d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)`

- num_features：一个样本的特征数量，等于上一个卷积层的输出通道数，这个参数最重要；**Batch Normalization 会在每一个特征维度上计算所有样本（样本数量为 batch_size 大小）的 runninng_mean 均值、running_var 方差、weight 和 bias 这四个参数。**
- eps：在进行标准化操作时的分母修正项，默认值很小，是为了避免分母为 0。
- momentum：估计当前均值和方差的加权系数。
- affine：是否需要 affine transform，默认为 True。
- track_running_stats：True 为训练状态，此时均值和方差会根据每个 mini-batch 加权计算。False 为测试状态，此时均值和方差会当前数据的统计值。

<br/>

**7. runninng_mean 均值和 running_var 方差的计算：**

1) 训练时：采用指数加权平均计算
- runninng_mean = (1-momentum) * pre_runninng_mean + momentum * mean_t
- running_var = (1-momentum) * pre_runninng_var + momentum * var_t

momentum 为记忆周期系数，pre_runninng_mean 为上一批次的均值，mean_t 为当前批次的均值，pre_runninng_var 为上一批次的方差，mean_t 为当前批次的方差

2) 测试时：当前数据的统计值

<br/>

**8. 在 PyTorch 中，有 3 个 Batch Normalization 类：**
- `bn = nn.BatchNorm1d(num_features=num_features, momentum=momentum)`，输入数据的形状是 batch_size * num_features * features_shape
- `bn = nn.BatchNorm2d(num_features=num_features, momentum=momentum)`，输入数据的形状是 batch_size * num_features * (features_length * features_width)
- `bn = nn.BatchNorm3d(num_features=num_features, momentum=momentum)`，输入数据的形状是 batch_size * num_features * (features_channels * features_length * features_width)

举例：
- `batch_size = 3, num_features = 5, features_shape = (1)` 表示输入数据的形状为 $3*5*1$，表示一个 mini-batch 有 3 个样本，每个样本有 5 个特征，每个特征的维度是 1，那么就会计算 5 组【均值、方差、weight 和 bias】，分别对应每个特征维度。
- `batch_size = 3, num_features = 3, features_shape = (2, 2)` 表示输入数据的形状为 $3*3*2*2$，表示一个 mini-batch 有 3 个样本，每个样本有 3 个特征，每个特征的维度是 $2*2$。
- `batch_size = 3, num_features = 3, features_shape = (3, 2, 2)` 表示输入数据的形状为 $3*3*3*2*2$，表示一个 mini-batch 有 3 个样本，每个样本有 3 个特征，每个特征的维度是 $3*2*2$。

<br/>

**9. 应用：**

**无 BN 层，使用 Kaiming 方法初始化权值：**

**注意：Batch Normalization 层一般在激活函数层的前一层。**

In [2]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)  # 使用 Kaiming 方法初始化权值

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.8343690633773804
layer:1, std:0.8857383728027344
layer:2, std:0.8768216371536255
layer:3, std:0.8620700836181641
layer:4, std:0.8756188750267029
layer:5, std:0.8784897923469543
layer:6, std:0.8935869336128235
layer:7, std:0.8712248206138611
layer:8, std:0.9630030989646912
layer:9, std:0.9609673619270325
layer:10, std:0.8539234399795532
layer:11, std:0.8073139190673828
layer:12, std:0.9279561638832092
layer:13, std:0.9541431069374084
layer:14, std:0.9491678476333618
layer:15, std:1.0219509601593018
layer:16, std:0.8949427604675293
layer:17, std:0.916878342628479
layer:18, std:0.8786735534667969
layer:19, std:0.9656010270118713
tensor([[0.0000, 0.1963, 1.8828,  ..., 0.0000, 0.0000, 1.6750],
        [0.0000, 0.0000, 1.3597,  ..., 0.0000, 0.0000, 1.2228],
        [0.0000, 0.0000, 2.4619,  ..., 0.0000, 0.0000, 2.1729],
        ...,
        [0.0000, 0.3190, 0.8542,  ..., 0.0000, 0.0000, 1.6953],
        [0.0000, 0.0305, 1.7972,  ..., 0.0000, 0.0000, 1.9260],
        [0.0000, 0

**有 BN 层，将权值直接初始化为标准正态分布：**

In [3]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.bn = nn.BatchNorm1d(num_features=neural_nums)
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = self.bn(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data)

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.5778297185897827
layer:1, std:0.5812498927116394
layer:2, std:0.581561267375946
layer:3, std:0.5803059339523315
layer:4, std:0.589631974697113
layer:5, std:0.5759917497634888
layer:6, std:0.5750452280044556
layer:7, std:0.5770913362503052
layer:8, std:0.5815293788909912
layer:9, std:0.5840634703636169
layer:10, std:0.5836883187294006
layer:11, std:0.5750855803489685
layer:12, std:0.5757170915603638
layer:13, std:0.5789809226989746
layer:14, std:0.5793242454528809
layer:15, std:0.5792165994644165
layer:16, std:0.5756219029426575
layer:17, std:0.5778035521507263
layer:18, std:0.5831732153892517
layer:19, std:0.5793583989143372
tensor([[0.5744, 0.0000, 0.0567,  ..., 2.1843, 0.2327, 0.7803],
        [0.6528, 0.0000, 0.0000,  ..., 0.0000, 0.3646, 0.2953],
        [0.0000, 0.0000, 0.0000,  ..., 0.9208, 0.0000, 0.0000],
        ...,
        [0.9975, 0.0000, 0.4790,  ..., 0.0099, 0.0819, 0.0000],
        [0.0000, 0.1138, 0.0000,  ..., 0.0000, 0.6335, 0.0000],
        [1.2653, 0.

**无 BN 层，将权值直接初始化为标准正态分布：**

In [4]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data)

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:9.43980884552002
layer:1, std:113.37451934814453
layer:2, std:1269.7734375
layer:3, std:14124.1572265625
layer:4, std:162308.0625
layer:5, std:1842326.75
layer:6, std:21201752.0
layer:7, std:233867664.0
layer:8, std:2924641280.0
layer:9, std:33018591232.0
layer:10, std:331950915584.0
layer:11, std:3550604689408.0
layer:12, std:46173452763136.0
layer:13, std:537134884716544.0
layer:14, std:6045300149977088.0
layer:15, std:7.363933600376422e+16
layer:16, std:7.29592073165996e+17
layer:17, std:8.456711715864707e+18
layer:18, std:9.16900778393102e+19
layer:19, std:1.1399807629213187e+21
tensor([[0.0000e+00, 2.3178e+20, 2.2228e+21,  ..., 0.0000e+00, 0.0000e+00,
         1.9774e+21],
        [0.0000e+00, 0.0000e+00, 1.6053e+21,  ..., 0.0000e+00, 0.0000e+00,
         1.4436e+21],
        [0.0000e+00, 0.0000e+00, 2.9065e+21,  ..., 0.0000e+00, 0.0000e+00,
         2.5653e+21],
        ...,
        [0.0000e+00, 3.7656e+20, 1.0084e+21,  ..., 0.0000e+00, 0.0000e+00,
         2.0014e+2

**有 BN 层，使用 Kaiming 方法初始化权值：**

In [5]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.bn = nn.BatchNorm1d(num_features=neural_nums)
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = self.bn(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)  # 使用 Kaiming 方法初始化权值

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.5778279304504395
layer:1, std:0.5812448859214783
layer:2, std:0.5815566182136536
layer:3, std:0.58030104637146
layer:4, std:0.5896270275115967
layer:5, std:0.5759868025779724
layer:6, std:0.5750400424003601
layer:7, std:0.5770862102508545
layer:8, std:0.5815243721008301
layer:9, std:0.5840585827827454
layer:10, std:0.5836827158927917
layer:11, std:0.5750799179077148
layer:12, std:0.5757117867469788
layer:13, std:0.578975260257721
layer:14, std:0.5793191194534302
layer:15, std:0.5792108178138733
layer:16, std:0.5756176710128784
layer:17, std:0.5777992010116577
layer:18, std:0.5831680297851562
layer:19, std:0.5793556571006775
tensor([[0.5735, 0.0000, 0.0570,  ..., 2.1840, 0.2318, 0.7803],
        [0.6528, 0.0000, 0.0000,  ..., 0.0000, 0.3644, 0.2948],
        [0.0000, 0.0000, 0.0000,  ..., 0.9206, 0.0000, 0.0000],
        ...,
        [0.9971, 0.0000, 0.4790,  ..., 0.0098, 0.0819, 0.0000],
        [0.0000, 0.1136, 0.0000,  ..., 0.0000, 0.6338, 0.0000],
        [1.2664, 0.0

**总结：**
- 无 BN 层，使用 Kaiming 方法初始化权值：输出值的标准差很小
- 有 BN 层，将权值直接初始化为标准正态分布：输出值的标准差很小
- 无 BN 层，将权值直接初始化为标准正态分布：输出值的标准差越来越大，数据尺度越来越大
- 有 BN 层，使用 Kaiming 方法初始化权值：输出值的标准差很小

**【添加 BN 层】和【使用 Kaiming 方法初始化权值】这两种方法使用其一就能很好地控制输出值的数据尺度；两者都用也可以；两者都不用时，输出值的数据尺度越来越大。**

以上针对的是非饱和激活函数 relu，饱和激活函数使用 Xavier 方法，同理。